In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

0 - No such habit
1- habit currently practiced
2 - past habit now has stopped (minimum 6 months)

In [ ]:
lfall.filter(pl.col("ID") == "test").select(
  pl.col("DATE REFERRED QUIT SER", "TARIKH TEMUJANJI QUIT SERVICE"),
  pl.when(pl.col("DATE REFERRED QUIT SER").is_not_null())
  .then(True)
  .otherwise(False)
  .alias("date_referred_filled"),
  pl.when(pl.col("TARIKH TEMUJANJI QUIT SERVICE").is_not_null())
  .then(True)
  .otherwise(False)
  .alias("appt_date_filled"),
  pl.when(pl.col("TARIKH TEMUJANJI QUIT SERVICE") >= pl.col("DATE REFERRED QUIT SER"))
  .then(True)
  .otherwise(False)
  .alias("valid_date_sequence"),
).filter(pl.any_horizontal(["date_referred_filled", "appt_date_filled"])).with_columns(
  (pl.col("date_referred_filled") == pl.col("appt_date_filled")).alias(
    "valid_completeness"
  ),
  pl.col("valid_date_sequence"),
).filter(~pl.all_horizontal("valid_completeness", "valid_date_sequence")).collect()
